In [75]:
import os
import gensim
from gensim.models.doc2vec import TaggedDocument
import numpy as np
import collections
from datetime import datetime
from datetime import timedelta
from sklearn.decomposition import TruncatedSVD
from time import time
from pathlib import Path
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
SPACY_tokenizer = Tokenizer(nlp.vocab)
from nltk.corpus import stopwords
nltk_stop_words = stopwords.words('english')
from pandarallel import pandarallel
from  tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
import re
import spacy

pandarallel.initialize()
from collections import OrderedDict
SPACY_NLP = spacy.load('en_core_web_lg')

def get_taggedDocCorpus(list_docs, list_ids):
    return TaggedDocument(list_docs, list_ids)


def build_model(list_docs, list_ids, epochs = 100):
    model = gensim.models.doc2vec.Doc2Vec(vector_size=256, hs = 1, min_count= 3, epochs=100)
    
    documents = [TaggedDocument(doc, [i]) for doc, i in zip(list_docs, list_ids)]
    print(documents)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    return model.docvecs, model

def get_docVector(modelObj, doc):
    vector = modelObj.infer_vector(doc)
    return vector

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [71]:
# ranks = []
# second_ranks = []
# for doc_id in range(len(train_corpus)):
#     inferred_vector = model.infer_vector(train_corpus[doc_id].words)
#     sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
#     rank = [docid for docid, sim in sims].index(doc_id)
#     ranks.append(rank)

#     second_ranks.append(sims[1])
# counter = collections.Counter(ranks)
# print(counter)

In [72]:
docs = [['activists', 'urge', 'la', 'police', 'arrest', 'soho', 'karen', 'seen', 'viral', 'video.', 'north', 'hollywood', 'calif.', 'activists', 'gathered', 'front', 'los', 'angeles', 'police', 'department`s', 'north', 'hollywood', 'station', 'saturday', 'urge', 'police', 'arrest', 'local', 'woman', 'known', 'social', 'media', 'soho', 'karen.', '22-year-old', 'miya', 'ponsetto', 'seen', 'video', 'accusing', '14-year-old', 'son', 'jazz', 'trumpeter', 'keyon', 'harrold', 'stealing', 'iphone.', '   ', 'racially', 'motivated', 'attack', 'ponsetto', 'tackled', 'boy', 'manhattan', 'hotel', 'lobby', 'later', 'turned', 'cellphone.', 'folks', 'los', 'angeles', 'say', 'ponsetto', 'may', 'involved', 'least', 'one', 'local', 'hotel', 'altercation.', '  ', 'beverly', 'hills', 'police', 'confirm', 'ponsetto', 'mother', 'arrested', 'peninsula', 'hotel', 'february', '2020', 'refused', 'leave', 'officers', 'called', 'remove', 'them.', 'ponsetto', 'arrested', 'drunk', 'public', 'mom', 'also', 'faces', 'charges', 'assaulting', 'police', 'officer.', 'seems', 'time', 'caught', 'video', 'arlo', 'soho', 'ny', 'ponsetto', 'also', 'parole', 'pleading', 'contest', 'local', 'dui', 'arrest', 'may', '2020.', 'lost', 'hills', 'sheriff', 'confirm', 'woman.', '  ', 'arlo', 'security', 'images', 'released', 'nypd', 'show', 'following', 'tackling', '14-year-old', 'prevent', 'leaving.', 'phone', 'eventually', 'returned', 'uber', 'driver', 'reportedly', 'said', 'left', 'behind', 'vehicle.', 'new', 'york', 'detectives', 'say', 'want', 'find', 'her.', 'able', 'reach', 'family', 'members', 'comment.'], ['america', 'go', 'crazy', '2020', 'america', 'go', 'crazy', '2020', 'suspect', 'observers', 'years', 'hence', 'think', 'responses', 'elite', 'officials', 'ordinary', 'americans', 'covid-19', 'pandemic', 'starting', 'last', 'february', 'shocking', 'video', 'minneapolis', 'police', 'officers', 'released', 'memorial', 'day', 'weekend.', 'response', 'covid', 'unprecedented', 'disproportionate', 'threat.', 'initially', 'pandemic', 'evoked', 'memories', 'ebola', 'sars', 'diseases', 'readily', 'contagious', 'fatal', 'roughly', 'half', 'infected.', 'intensive', 'testing', 'contact', 'tracing', 'quarantines', 'indicated', 'initially', 'hailed', 'effective', 'island', 'countries', '--', 'taiwan', 'singapore', 'south', 'korea', 'new', 'zealand.', 'continent-sized', 'globally', 'connected', 'united', 'states', 'disease', 'highly', 'contagious', 'often', 'asymptomatic', 'could', 'stamped', 'way.', 'respiratory', 'ailment', 'lower', 'fatality', 'rate', 'among', '65', 'bad', 'year`s', 'influenza', 'surely', 'didn`t', 'justify', 'extended', 'lockdowns', 'restrictions', 'others', '(no', 'garden', 'seeds', 'threw', 'economy', 'sharp', 'reversal', 'devastated', 'many', 'low-wage', 'workers.', 'worries', 'proved', 'unwarranted', 'overwhelming', 'icus', 'resulted', 'responses', 'new', 'york', 'gov.', 'andrew', 'cuomo`s', 'order', 'sending', 'covid', 'patients', 'assisted', 'living', 'facilities', 'deadly', 'mistakes.', 'covid', 'split', 'americans', 'partisan', 'lines', 'democrats', 'pushing', 'republicans', 'pushing', 'strict', 'lockdowns.', 'experts', 'lionized', 'legacy', 'media', 'often', 'help', 'disparaging', 'international', 'travel', 'bans', 'xenophobia', 'endorsing', 'interstate', 'travel', 'bans', 'saying', 'people', 'shouldn`t', 'wear', 'masks', 'must', 'bashing', 'people', 'thronged', 'wind-swept', 'beaches', 'cheering', 'others', 'thronging', 'tightly', 'packed', 'anti-police', 'demonstrations.', '`mostly', 'peaceful`', 'demonstrations', 'protests', 'arrestee`s', 'death', 'hands', 'minneapolis', 'police.', 'incident', 'seems', 'convinced', 'millions', 'americans', 'encouragement', 'legacy', 'media', 'nation', 'faced', 'sudden', 'upsurge', 'white', 'cops', 'killing', 'innocent', 'blacks.', 'statistics', 'tell', 'another', 'story.', 'washington', 'post`s', 'chronicle', 'begun', 'black', 'lives', 'matter', 'movement', 'exploited', '2014', 'ferguson', 'missouri', 'case', 'showed', 'number', 'police', 'shootings', 'black', 'suspects', 'declining', 'proportion', 'far', 'lower', 'proportion', 'violent', 'crimes', 'committed', '(and', 'against)', 'blacks', 'manhattan', 'institute`s', 'heather', 'mac', 'donald', 'pointed', 'out.', 'democratic', 'politicians', 'rallied', 'join', 'blm', 'cause', 'kente', 'cloth-clad', 'congressional', 'leaders', 'kneeling', 'ceremony.', 'defunding', 'police', 'became', 'rallying', 'cry', 'public', 'policy', 'democratic-run', 'central', 'cities', '$282', 'million', 'police', 'budget', 'cuts', 'new', 'york', '$150', 'million', 'cut', 'los', 'angeles', '$69', 'million', 'seattle', '$60', 'million', 'san', 'francisco', '$50', 'million', 'denver.', 'de-policing', 'whether', 'defunding', 'cops', 'avoiding', 'proactive', 'policing', 'fear', 'career-ending', 'confrontations', 'results', 'violent', 'crime.', 'violent', 'crime', 'rose', 'cities', 'st.', 'louis', 'baltimore', 'blm`s', 'ferguson', 'protests', 'protests', '2019', '2020', 'murders', 'increased', '72%', 'minneapolis', '62%', 'new', 'york', '55%', 'chicago', '78%', 'louisville', '51%', 'portland.', 'murdered', 'way', 'black.', 'murder', 'rates', 'shot', 'memorial', 'day', 'final', 'data', 'isn`t', 'available', 'looks', 'like', 'murders', 'nationwide', '2019', 'least', '15%.', 'would', 'biggest', 'one-year', 'increase', 'ever', 'recorded.', '12.7%', '1968', '--', 'also', 'tumultuous', 'election', 'year', 'saw', 'multiple', 'urban', 'riots', 'horrifying', 'assassinations', 'martin', 'luther', 'king', 'jr.', 'robert', 'kennedy.', 'time', '`police', 'brutality`', 'far', 'common', 'midst', 'decade', '1965-75', 'violent', 'crime', 'roughly', 'tripled.', 'brought', 'proactive', 'policing', 'reforms', 'initiated', 'rudy', 'giuliani', 'new', 'york', 'pursued', 'many', 'others.', 'murders', 'new', 'york', 'fell', '200', '300', 'betwee']]

In [73]:
a = 'Killer of 11,000 Nursing Home Patients Calls Orthodox Wedding `Disrespectful` - Frontpagemag. Above is a screenshot of Governor Cuomo, along with Al Sharpton, and anti-police activists, at an event over the summer. The picture is a reminder that Cuomo is as committed to masking and social distancing as Governor Newsom. But Cuomo, unlike Newsom, also has a large Orthodox Jewish population he can scapegoat. And when the New York Post`s intrepid Orthodox Jewish investigators got hold of reports of one of those underground Jewish weddings, Cuomo got on the case. Its illegal. It was also disrespectful of the people of New York,` Cuomo, who has been blamed for the deaths of 11,000 senior citizens by forcing nursing home patients to accept infected patients, ranted. If it turns out that because we stopped that wedding the reaction was, `Well well have a secret wedding, that would be really shocking and totally deceitful.. The shocking part here is that it`s 2020 and. 1. New York Democrats and the media are scapegoating Orthodox Jews and traditional Christians for a virus. 2. A wedding is treated like a crime. 3. Cuomo still isn`t in jail. But, 5 minutes after the last Biden victory party and Black Lives Matter race riot, lefties can come together to fume at those horrible Chassidic Jews and their weddings. '
b = 'Media Unveils Bold New Plan to Win Latino Support by Calling Them Racist - Frontpagemag. The ruling class knows only two ways to communicate with its subjects. (Three ways if you count omnipotent claims to be speaking on behalf of science and truth.). 1. Call them racist . 2. Tell them they`re the victims of racism. Since President Trump boosted his Latino support in 2020 and Republicans actually flipped House seats with Latino support, the media has decided to embark on a bold new way to enlist Latino support. by accusing them of racism. For now, it`s doing so in a semi-sympathetic `you poor ignorant, dears` way that it used to talk about the white working class in the sixties. But this strategy doesn`t exactly bode well. Here`s Politico on Latinos and . `Parra, who dislikes the defund slogan but not its goal of stopping police violence, said Trumps improved standing with Latinos amid the protests reflects a little-discussed problem in Hispanic communities: anti-Blackness. Thats an opinion held by other Latino commentators as well Black Lives Matter demonstrators.`. 2 out of 3 racists agree. Opposing us is racist. `Anti-Blackness` by the way is the Orwellian term that replaced `racism`  because it can inconveniently be used to refer to black nationalists. While anti-blackness doesn`t even really refer to racism, but any opposition to black nationalist racism. Also, Latinos who vote Republican are officially `white Hispanics`. In Texas majority-Hispanic Rio Grande Valley and along the Texas border, where Trump did well for a Republican, progressive organizer Ofelia Alonso pointed out that `Latino` is a broad and imprecise catchall term for members of an ethnic group in which people identify as Black, white, indigenous, Asian, Middle Eastern or mixed race. A lot of people who voted for Trump, while theyre Latino, theyre also white, she said, pointing to the city of Harlingen as a Trump-supporting white city with money, . The strategy here is to stop treating Latinos as a bloc and to start microorganizing and racializing different portions of the demo, while attacking others. Here`s  ABC News. The most overarching and effective disinformation narrative, according to Jaime Longoria, an investigative researcher with First Draft News, was an effort to play on what he called an `anti-Blackness` tactic -- a part of the push of disinformation to Latino voters, particularly in Florida. Longoria said videos of confrontations shared on WhatsApp and Facebook pushed the notion that Black people were `harassing` Latinos under the guise of activism. He said much of this content played to prejudices. . Black Lives Matter racists harassing people. plays to prejudices. So the current plan is to accuse any Latinos who might be persuadable by Republicans of racism and white privilege, and putting them down on the low end of the victimhood totem pole?. Sounds like a great plan for recruiting more Republican voters. '
c = 'Black Lives Matter activist Frank Nitty released from jail without charges but investigation is still open. Milwaukee Black Lives Matter protest leader Frank Sensabaugh has been released from jail without charges. after a woman made sexual assault allegations against him. Sensabaugh, 39, better known as Frank Nitty II, livestreamed the encounter, which was civil, while explaining who he believed made the accusations. On Friday, Chief Deputy District Attorney Kent Lovern said his office was reviewing the matter and the investigation was continuing, but that Sensabaugh had been released from custody. '

In [76]:
def clean_text_3(text):
    global nltk_stop_words
    global SPACY_NLP
    global SPACY_tokenizer
    text = re.sub('[0-9]+', '', text)
    sentences = [s.__repr__().strip() for s in SPACY_NLP(text).sents][:10]
    sentences = [  SPACY_tokenizer.__call__(s) for s in sentences]
    
    s1 = [ [ str.lower(w.lemma_)  for w in  sent if len(w.lemma_)>1 and w.lemma_ not in nltk_stop_words ] for sent in sentences]
    text = []
    for _ in s1: text.extend(_)
    return text

In [77]:
r = [ clean_text_3(a), clean_text_3(b),clean_text_3(c)]

In [78]:
build_model(r, [1,2,3])

[TaggedDocument(words=['killer', 'nursing', 'home', 'patients', 'calls', 'orthodox', 'wedding', '`disrespectful`', 'frontpagemag.', 'above', 'screenshot', 'governor', 'cuomo,', 'along', 'al', 'sharpton,', 'anti-police', 'activists,', 'event', 'summer.', 'the', 'picture', 'reminder', 'cuomo', 'committed', 'masking', 'social', 'distancing', 'governor', 'newsom.', 'but', 'cuomo,', 'unlike', 'newsom,', 'also', 'large', 'orthodox', 'jewish', 'population', 'scapegoat.', 'and', 'new', 'york', 'post`s', 'intrepid', 'orthodox', 'jewish', 'investigators', 'got', 'hold', 'reports', 'one', 'underground', 'jewish', 'weddings,', 'cuomo', 'got', 'case.', 'its', 'illegal.', 'it', 'also', 'disrespectful', 'people', 'new', 'york,`', 'cuomo,', 'blamed', 'deaths', 'senior', 'citizens', 'forcing', 'nursing', 'home', 'patients', 'accept', 'infected', 'patients,', 'ranted.', 'if', 'turns', 'stopped', 'wedding', 'reaction', 'was,', '`well', 'well', 'secret', 'wedding,', 'would', 'really', 'shocking', 'totally

(<gensim.models.keyedvectors.Doc2VecKeyedVectors at 0x7f74a437cad0>,
 <gensim.models.doc2vec.Doc2Vec at 0x7f74a437ca50>)